# Notebook 04: MCP (Model Context Protocol)

## 🎯 What is This Notebook About?

Welcome to Notebook 04! In this notebook, we'll explore **MCP (Model Context Protocol)** - a protocol for integrating external tools and services with LLMs.

**What we'll learn:**
1. **What is MCP** - Understanding the Model Context Protocol
2. **Tool Execution** - How agents call and use tools
3. **Tool Integration Patterns** - Different ways to integrate tools
4. **Creating Custom Tools** - Building your own tools for agents

**Why this matters:**
- LLMs can't directly interact with systems
- MCP provides a standardized way to connect tools
- Enables agents to take real actions
- Makes agents more powerful and useful

---

## 📚 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what MCP is and why it's important
- ✅ Know how tools are executed by agents
- ✅ Learn different tool integration patterns
- ✅ Be able to create custom tools
- ✅ Understand how tools enable agents to take actions

---

## ⚙️ Prerequisites

- LlamaStack server running (see Module README)
- Ollama running with llama3.2:3b model
- Python environment with dependencies installed
- Understanding of Notebook 03 (Simple Chat and RAG)

---

## 🔧 Setup

Let's start by connecting to LlamaStack and verifying everything is working.


In [ ]:
# Import required libraries
import os
from llama_stack_client import LlamaStackClient

# Configuration
llamastack_url = os.getenv("LLAMA_STACK_URL", "http://localhost:8321")
model = os.getenv("LLAMA_MODEL", "ollama/llama3.2:3b")

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")

# Initialize LlamaStack client
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {len(models)}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("   Please ensure LlamaStack is running:")
    print("   python scripts/start_llama_stack.py")
    raise


## Part 1: What is MCP?

### What is MCP?

**MCP (Model Context Protocol)** is a protocol for integrating external tools and services with LLMs. It allows agents to:
- **Call external APIs** (e.g., check service status, restart services)
- **Access databases** (e.g., query incident logs)
- **Execute commands** (e.g., run system commands)
- **Integrate with other systems** (e.g., monitoring tools, ticketing systems)

**Why MCP matters:**
- LLMs can't directly interact with systems
- MCP provides a standardized way to connect tools
- Enables agents to take real actions
- Makes agents more powerful and useful

**When to use MCP:**
- Need to interact with external systems
- Want agents to take actions (not just answer questions)
- Need real-time data from APIs
- Want to integrate with existing tools

---

### Hands-on: Exploring Tool Runtime

Let's explore what tools are available and how they work.


In [ ]:
# Example 1: Understanding MCP Tools
print("=" * 60)
print("Example 1: Understanding MCP Tools")
print("=" * 60)

print("\n💡 MCP (Model Context Protocol) Tools:")
print("   - Allow agents to call external APIs")
print("   - Enable system command execution")
print("   - Provide database access")
print("   - Integrate with monitoring systems")
print("\n📝 In Notebook 02, we saw how to create custom tools.")
print("   Tools are Python functions that agents can call.")
print("\n💡 MCP provides a standardized protocol for tool integration.")
print("   Tools can be:")
print("   - Client-side (run in your Python process)")
print("   - Server-side (registered with LlamaStack)")
print("   - External APIs (via HTTP)")
print("\n✅ We'll see tool integration in action in Notebook 05!")


## Part 2: Understanding Tool Execution

Tools are functions that agents can call. When an agent needs to perform an action, it:
1. **Decides** which tool to use
2. **Calls** the tool with appropriate parameters
3. **Receives** the result
4. **Uses** the result to continue reasoning

**Tool Structure:**
- **Name**: Identifies the tool
- **Description**: Tells the LLM what the tool does
- **Parameters**: What inputs the tool needs
- **Returns**: What the tool outputs


In [ ]:
# Example 2: Create a simple custom tool
print("=" * 60)
print("Example 2: Creating a Custom Tool")
print("=" * 60)

# Define a simple tool function
def check_service_status(service_name: str) -> str:
    """
    Check the status of a system service.
    
    Args:
        service_name: Name of the service to check (e.g., 'nginx', 'mysql')
    
    Returns:
        Status of the service: 'running', 'stopped', or 'not found'
    """
    # Simulate service check (in practice, this would call systemctl or similar)
    import random
    statuses = ['running', 'stopped', 'not found']
    status = random.choice(statuses)
    
    return f"Service '{service_name}' is {status}."

# Test the tool
print("\n🔧 Custom Tool: check_service_status")
print("   Description: Check the status of a system service")
print("   Parameters: service_name (str)")
print("\n📝 Testing tool:")
result = check_service_status("nginx")
print(f"   check_service_status('nginx') → {result}")

print("\n💡 In Notebook 02, we saw how to use tools with agents.")
print("   Tools enable agents to take actions, not just answer questions.")


## Part 3: Creating an MCP Server for Terminal Access

Now let's create a practical MCP server that provides terminal access. This will demonstrate how MCP works in practice.

**What we'll build:**
- A simple MCP server that can execute terminal commands
- Safe command execution with basic validation
- Integration with LlamaStack to use terminal tools in agents

**Why this is useful:**
- Agents can execute system commands
- Useful for IT operations tasks
- Demonstrates real-world MCP usage


In [ ]:
# Example 3: Create a Simple MCP Server for Terminal Access
print("=" * 60)
print("Example 3: Creating an MCP Server for Terminal Access")
print("=" * 60)

import subprocess
import json
from typing import Any
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import Tool, TextContent

# Create MCP server instance
mcp_server = Server("terminal-mcp-server")

# Define safe commands (whitelist approach for security)
SAFE_COMMANDS = {
    "ls", "pwd", "whoami", "date", "uptime", 
    "df", "du", "free", "ps", "top", "htop",
    "cat", "head", "tail", "grep", "find",
    "echo", "uname", "hostname", "env"
}

def is_safe_command(command: str) -> bool:
    """Check if command is in the safe whitelist."""
    cmd_parts = command.strip().split()
    if not cmd_parts:
        return False
    base_cmd = cmd_parts[0]
    return base_cmd in SAFE_COMMANDS

@mcp_server.list_tools()
async def list_tools() -> list[Tool]:
    """List available tools."""
    return [
        Tool(
            name="execute_terminal_command",
            description="Execute a safe terminal command. Supported commands: ls, pwd, whoami, date, uptime, df, du, free, ps, top, cat, head, tail, grep, find, echo, uname, hostname, env. Returns command output or error message.",
            inputSchema={
                "type": "object",
                "properties": {
                    "command": {
                        "type": "string",
                        "description": "The terminal command to execute (must be from safe whitelist)"
                    }
                },
                "required": ["command"]
            }
        )
    ]

@mcp_server.call_tool()
async def call_tool(name: str, arguments: Any) -> list[TextContent]:
    """Handle tool calls."""
    if name == "execute_terminal_command":
        command = arguments.get("command", "")
        
        if not command:
            return [TextContent(
                type="text",
                text="Error: No command provided"
            )]
        
        if not is_safe_command(command):
            return [TextContent(
                type="text",
                text=f"Error: Command '{command.split()[0]}' is not in the safe whitelist. Allowed commands: {', '.join(sorted(SAFE_COMMANDS))}"
            )]
        
        try:
            # Execute command safely
            result = subprocess.run(
                command,
                shell=True,
                capture_output=True,
                text=True,
                timeout=10,
                cwd=None  # Use current directory
            )
            
            output = result.stdout if result.stdout else result.stderr
            return_code = result.returncode
            
            response = f"Command: {command}\n"
            response += f"Return code: {return_code}\n"
            response += f"Output:\n{output}"
            
            return [TextContent(type="text", text=response)]
            
        except subprocess.TimeoutExpired:
            return [TextContent(
                type="text",
                text=f"Error: Command '{command}' timed out after 10 seconds"
            )]
        except Exception as e:
            return [TextContent(
                type="text",
                text=f"Error executing command: {str(e)}"
            )]
    
    return [TextContent(type="text", text=f"Unknown tool: {name}")]

print("\n✅ MCP server defined!")
print("   Tools available:")
print("   - execute_terminal_command: Execute safe terminal commands")
print(f"\n   Safe commands: {', '.join(sorted(SAFE_COMMANDS))}")
print("\n💡 This server can be started and connected to LlamaStack.")
print("   We'll see how to use it in the next example.")


## Part 4: The MCP Server File

The MCP server code is already available in the repository at `5-autonomous-agents/src/mcp_terminal_server.py`. 

**What the server provides:**
- Safe terminal command execution
- Command whitelist for security
- Timeout protection
- Error handling

**Key features:**
- Whitelist of safe commands (ls, pwd, whoami, date, etc.)
- Validates commands before execution
- Returns structured results with return codes and output


In [ ]:
# Example 4: Show MCP Server File Location
print("=" * 60)
print("Example 4: MCP Server File")
print("=" * 60)

import os
from pathlib import Path

# Get the workspace root (assuming notebook is in notebooks/ subdirectory)
notebook_dir = Path.cwd()
# If we're in notebooks/, go up one level, then to src/
if notebook_dir.name == 'notebooks':
    workspace_root = notebook_dir.parent
else:
    # Fallback: assume we're at workspace root
    workspace_root = notebook_dir

mcp_server_file = workspace_root / "src" / "mcp_terminal_server.py"

if mcp_server_file.exists():
    print(f"\n✅ MCP server file found: {mcp_server_file}")
    print(f"   File size: {mcp_server_file.stat().st_size} bytes")
    
    # Show first few lines to confirm it's the right file
    with open(mcp_server_file, 'r') as f:
        lines = f.readlines()[:10]
        print(f"\n📄 First few lines of the server file:")
        for i, line in enumerate(lines, 1):
            print(f"   {i:2d}: {line.rstrip()}")
    
    print(f"\n💡 The MCP server is ready to use!")
    print(f"   You can start it in the next cell.")
else:
    print(f"\n⚠️  MCP server file not found: {mcp_server_file}")
    print(f"   Please ensure the file exists in the repository.")


## Part 4.5: Running the MCP Server in the Notebook

Let's start the MCP server directly from this notebook. This way you don't need to run it in a separate terminal.

**Note:** The MCP server will run in the background. You can stop it later if needed.


In [ ]:
# Example 4.5: Start MCP Server in Background
print("=" * 60)
print("Example 4.5: Starting MCP Server")
print("=" * 60)

import subprocess
import sys
import time
import os
from pathlib import Path

# Get the workspace root (assuming notebook is in notebooks/ subdirectory)
notebook_dir = Path.cwd()
# If we're in notebooks/, go up one level, then to src/
if notebook_dir.name == 'notebooks':
    workspace_root = notebook_dir.parent
else:
    # Fallback: assume we're at workspace root
    workspace_root = notebook_dir

mcp_server_file = workspace_root / "src" / "mcp_terminal_server.py"

if not mcp_server_file.exists():
    print("⚠️  MCP server file not found.")
    print(f"   Expected location: {mcp_server_file}")
else:
    print(f"✅ Found MCP server file: {mcp_server_file}")
    
    # Check if server is already running
    mcp_process = None
    
    try:
        # Start the MCP server as a subprocess
        # Note: MCP servers communicate via stdio, so we need to handle stdin/stdout
        print("\n🚀 Starting MCP server...")
        
        mcp_process = subprocess.Popen(
            [sys.executable, str(mcp_server_file)],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1
        )
        
        # Give it a moment to start
        time.sleep(1)
        
        # Check if process is still running
        if mcp_process.poll() is None:
            print("✅ MCP server started successfully!")
            print(f"   Process ID: {mcp_process.pid}")
            print("\n💡 The server is now running in the background.")
            print("   LlamaStack can connect to it via stdio.")
            print("\n⚠️  Note: To stop the server, you can:")
            print(f"   - Call mcp_process.terminate()")
            print(f"   - Or restart the kernel")
        else:
            # Process exited immediately - check for errors
            stdout, stderr = mcp_process.communicate()
            print(f"❌ MCP server failed to start")
            if stderr:
                print(f"   Error: {stderr[:200]}")
            if stdout:
                print(f"   Output: {stdout[:200]}")
            mcp_process = None
            
    except Exception as e:
        print(f"❌ Error starting MCP server: {e}")
        import traceback
        traceback.print_exc()
        mcp_process = None

# Store the process in a global variable for later use
if 'mcp_process' in locals() and mcp_process:
    print(f"\n📝 To stop the server later, run:")
    print(f"   mcp_process.terminate()")


## Part 5: Testing the MCP Server Directly

Before using it with agents, let's test the MCP server directly to make sure it works. We'll simulate a tool call to verify the server responds correctly.

Once the MCP server is running and connected to LlamaStack, agents can use the terminal tools. Let's see how this works conceptually.

**How it works:**
1. MCP server exposes tools (like `execute_terminal_command`)
2. LlamaStack connects to the MCP server
3. Agents can call these tools when needed
4. Tools execute and return results to the agent

**Example workflow:**
- User asks: "What's the current directory?"
- Agent decides to use `execute_terminal_command` with `pwd`
- Tool executes and returns the directory path
- Agent responds with the result


In [ ]:
# Example 5: Test MCP Server Directly
print("=" * 60)
print("Example 5: Testing MCP Server")
print("=" * 60)

# Since MCP servers communicate via stdio protocol, testing directly is complex
# Instead, let's demonstrate the server functionality by testing the command execution logic

import subprocess

print("\n🧪 Testing command execution logic (simulating MCP tool call):\n")

test_commands = [
    "pwd",
    "whoami",
    "date",
    "ls -la | head -5"  # This should work since 'ls' is in the whitelist
]

SAFE_COMMANDS = {
    "ls", "pwd", "whoami", "date", "uptime", 
    "df", "du", "free", "ps", "top", "htop",
    "cat", "head", "tail", "grep", "find",
    "echo", "uname", "hostname", "env"
}

def is_safe_command(command: str) -> bool:
    """Check if command is in the safe whitelist."""
    cmd_parts = command.strip().split()
    if not cmd_parts:
        return False
    base_cmd = cmd_parts[0]
    return base_cmd in SAFE_COMMANDS

for cmd in test_commands:
    print(f"🔍 Testing: {cmd}")
    
    if not is_safe_command(cmd):
        print(f"   ❌ Command '{cmd.split()[0]}' is not in safe whitelist")
        continue
    
    try:
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=5
        )
        
        output = result.stdout.strip() if result.stdout else result.stderr.strip()
        print(f"   ✅ Return code: {result.returncode}")
        print(f"   📤 Output: {output[:100]}{'...' if len(output) > 100 else ''}")
        
    except subprocess.TimeoutExpired:
        print(f"   ⏱️  Command timed out")
    except Exception as e:
        print(f"   ❌ Error: {e}")
    
    print()

print("✅ Command execution logic works correctly!")
print("\n💡 In a real scenario, LlamaStack would:")
print("   1. Connect to the MCP server via stdio")
print("   2. Call the execute_terminal_command tool")
print("   3. Receive the results")
print("   4. Use them in agent responses")


## Part 6: Using MCP Tools with Agents

Once the MCP server is running and connected to LlamaStack, agents can use the terminal tools. Let's see how this works conceptually.

**How it works:**
1. MCP server exposes tools (like `execute_terminal_command`)
2. LlamaStack connects to the MCP server
3. Agents can call these tools when needed
4. Tools execute and return results to the agent

**Example workflow:**
- User asks: "What's the current directory?"
- Agent decides to use `execute_terminal_command` with `pwd`
- Tool executes and returns the directory path
- Agent responds with the result


In [ ]:
# Example 6: Simulate Using MCP Tools (Conceptual)
print("=" * 60)
print("Example 6: Using MCP Tools with Agents (Conceptual)")
print("=" * 60)

print("\n💡 How an agent would use MCP terminal tools:\n")

# Simulate agent reasoning
scenarios = [
    {
        "user_query": "What's the current directory?",
        "agent_reasoning": "I need to check the current directory. I'll use execute_terminal_command with 'pwd'.",
        "tool_call": {"tool": "execute_terminal_command", "arguments": {"command": "pwd"}},
        "tool_result": "/Users/gsampaio/redhat/ai/hello-there-ai-ops-workshop",
        "agent_response": "The current directory is /Users/gsampaio/redhat/ai/hello-there-ai-ops-workshop"
    },
    {
        "user_query": "List files in the current directory",
        "agent_reasoning": "The user wants to see files. I'll use execute_terminal_command with 'ls -la'.",
        "tool_call": {"tool": "execute_terminal_command", "arguments": {"command": "ls -la"}},
        "tool_result": "total 48\ndrwxr-xr-x  5 user  staff  160 Nov 19 10:00 .\ndrwxr-xr-x  3 user  staff   96 Nov 19 09:00 ..\n-rw-r--r--  1 user  staff  1234 Nov 19 10:00 README.md\n...",
        "agent_response": "Here are the files in the current directory:\n[lists files]"
    },
    {
        "user_query": "Check system uptime",
        "agent_reasoning": "I need to check system uptime. I'll use execute_terminal_command with 'uptime'.",
        "tool_call": {"tool": "execute_terminal_command", "arguments": {"command": "uptime"}},
        "tool_result": "10:30:45 up 5 days,  2:15,  3 users,  load average: 0.50, 0.45, 0.40",
        "agent_response": "The system has been up for 5 days, 2 hours, and 15 minutes. Current load average is 0.50, 0.45, 0.40."
    }
]

for i, scenario in enumerate(scenarios, 1):
    print(f"\n{'='*60}")
    print(f"Scenario {i}: {scenario['user_query']}")
    print(f"{'='*60}")
    print(f"\n👤 User: {scenario['user_query']}")
    print(f"\n🤖 Agent Reasoning: {scenario['agent_reasoning']}")
    print(f"\n🔧 Tool Call:")
    print(f"   Tool: {scenario['tool_call']['tool']}")
    print(f"   Arguments: {scenario['tool_call']['arguments']}")
    print(f"\n📤 Tool Result:")
    print(f"   {scenario['tool_result'][:100]}...")
    print(f"\n💬 Agent Response: {scenario['agent_response']}")

print("\n\n✅ This demonstrates how agents can use MCP tools to:")
print("   - Execute terminal commands")
print("   - Get real-time system information")
print("   - Perform IT operations tasks")
print("   - Take actions, not just answer questions")


## Summary

### Key Takeaways

1. **MCP** provides a standardized protocol for tool integration
2. **Tools** enable agents to take actions, not just answer questions
3. **Tool execution** follows a clear pattern: decide → call → receive → use
4. **MCP servers** can provide terminal access and other capabilities
5. **Security** is important - use whitelists and validation for command execution

### When to Use MCP Tools

**Use MCP when:**
- ✅ Need to interact with external systems
- ✅ Want agents to take actions
- ✅ Need real-time data from APIs
- ✅ Want to integrate with existing tools
- ✅ Need terminal/system command execution

**Don't use MCP when:**
- ❌ Pure text generation is sufficient
- ❌ No external systems to interact with
- ❌ Simple Q&A without actions needed

### Security Considerations

When creating MCP servers for terminal access:
- ✅ Use command whitelists
- ✅ Validate all inputs
- ✅ Set timeouts on command execution
- ✅ Run with minimal privileges
- ✅ Log all command executions
- ✅ Consider sandboxing for production

### Next Steps

In **Notebook 05**, we'll explore **Safety** features, and in **Notebook 06**, we'll see how to **evaluate** agents using multiple metrics.

**Ready to learn about safety?** Let's move to Notebook 05!
